In [8]:
%%capture
!pip install chromadb
!pip install datasets
!pip install pyarrow
!pip install open-clip-torch
!pip install sentence-transformers
!pip install langchain_core langchain_openai

In [9]:
import IPython
from IPython.display import HTML, display, Image, Markdown, Video, Audio
from typing import Optional, Sequence, List, Dict, Union

import soundfile as sf

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

from google.colab import userdata

from sentence_transformers import SentenceTransformer
from transformers import ClapModel, ClapProcessor
from datasets import load_dataset

import chromadb
from chromadb.utils.embedding_functions import OpenCLIPEmbeddingFunction
from chromadb.utils.data_loaders import ImageLoader
from chromadb.api.types import Document, Embedding, EmbeddingFunction, URI, DataLoader

import numpy as np
import torchaudio
import base64
import torch
import json
import cv2
import os

In [10]:
def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [11]:
path = "mm_vdb"
client = chromadb.PersistentClient(path=path)

In [14]:
ds = load_dataset("TopicNavi/Wikipedia-example-data")

README.md:   0%|          | 0.00/571 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/135M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

model = SentenceTransformer('all-MiniLM-L6-v2').to(device)

documents = [entry['text'] for entry in ds['train']]
metadatas = [{"url": entry['url'], "wiki_id": entry['wiki_id']} for entry in ds['train']]
ids = [entry['title'] for entry in ds['train']]

embeddings = []
batch_size = 128

for i in range(0, len(documents), batch_size):
    batch = documents[i:i+batch_size]
    batch_embeddings = model.encode(batch, convert_to_tensor=True, device=device)
    embeddings.extend(batch_embeddings.cpu().numpy())

embeddings = [emb.tolist() for emb in embeddings]

export_data = {
    "documents": documents,
    "embeddings": embeddings,
    "metadatas": metadatas,
    "ids": ids
}

with open('wikipedia_embeddings.json', 'w') as f:
    json.dump(export_data, f)

print("Data exported to wikipedia_embeddings.json")

Using device: cuda


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Data exported to wikipedia_embeddings.json


In [16]:
def load_wiki(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)

    # Extract the components
    documents = data['documents']
    embeddings = data['embeddings']
    metadatas = data['metadatas']
    ids = data['ids']

    print(f"Loaded data from {file_path}")
    print(f"Number of entries: {len(documents)}")
    print(f"Embedding dimension: {len(embeddings[0])}")

    return documents, embeddings, metadatas, ids

docs, embs, metas, ids = load_wiki('wikipedia_embeddings.json')

Loaded data from wikipedia_embeddings.json
Number of entries: 25000
Embedding dimension: 384


In [17]:
text_collection = client.get_or_create_collection(name="text_collection")

In [18]:
def batch_add_to_collection(collection, documents, embeddings, metadatas, ids, batch_size=5461):
    for i in range(0, len(documents), batch_size):
        # Slice the data into batches
        doc_batch = documents[i:i + batch_size]
        emb_batch = embeddings[i:i + batch_size]
        meta_batch = metadatas[i:i + batch_size]
        id_batch = ids[i:i + batch_size]

        # Add the batch to the collection
        collection.add(
            documents=doc_batch,
            embeddings=emb_batch,
            metadatas=meta_batch,
            ids=id_batch
        )
        print(f"Batch {i // batch_size + 1} added to the collection successfully.")

batch_add_to_collection(text_collection, docs, embs, metas, ids)

Batch 1 added to the collection successfully.
Batch 2 added to the collection successfully.
Batch 3 added to the collection successfully.
Batch 4 added to the collection successfully.
Batch 5 added to the collection successfully.


In [23]:
def display_text_documents(query_text, max_distance=None, debug=False):
    results = text_collection.query(
        query_texts=[query_text],
        n_results=5,
        include=['documents', 'distances', 'metadatas']
    )

    documents = results['documents'][0]
    distances = results['distances'][0]
    metadatas = results['metadatas'][0]
    titles = results['ids'][0]

    for title, doc, distance, metadata in zip(titles, documents, distances, metadatas):
        url = metadata.get('url')

        if max_distance is None or distance <= max_distance:
            print(f"Title: {title.replace('_', ' ')}")
            if debug:
              print(f"Distance: {distance}")
            print(f"URL: {url}")
            print(f"Text: {doc}\n")
        else:
            if debug:
              print(f"Title: {title.replace('_', ' ')} - Distance: {distance} (Filtered out)")

display_text_documents("laptop", max_distance=1.3, debug=True)

Title: Laptop
Distance: 0.6660436391830444
URL: https://en.wikipedia.org/wiki?curid=198584
Text: A laptop, laptop computer, or notebook computer is a small, portable personal computer (PC) with a screen and alphanumeric keyboard. Laptops typically have a clam shell form factor with the screen mounted on the inside of the upper lid and the keyboard on the inside of the lower lid, although 2-in-1 PCs with a detachable keyboard are often marketed as laptops or as having a "laptop mode". Laptops are folded shut for transportation, and thus are suitable for mobile use. They are so named because they can be practically placed on a person's lap when being used. Today, laptops are used in a variety of settings, such as at work, in education, for playing games, web browsing, for personal multimedia, and for general home computer use.

Title: Lenovo
Distance: 1.0233261585235596
URL: https://en.wikipedia.org/wiki?curid=997189
Text: Lenovo Group Limited, often shortened to Lenovo ( , ), is a Chine